<a href="https://colab.research.google.com/github/golamshaifullah/EPTADR2_tutorial/blob/main/tutorials/02_Enterprise_SinglePSR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pulsar noise modelling tutorial - adapted from notebooks by Aurelien Chalumeau and Irene Ferranti (UniMiB)

### Run the following two cells only when using colab! 

In [ ]:
# This cell will reset the kernel.
# Run this cell, wait until it's done, then run the next.
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge()

In [ ]:
%%capture
!mamba install -y -c conda-forge enterprise_extensions la_forge corner "scipy<1.13"
!git clone https://github.com/golamshaifullah/EPTADR2_tutorial

### The actual notebook starts from here:

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    homedir = '/content/EPTADR2_tutorial'
else:
    homedir = '../'

### GitHub Repositories:
- **libstempo**: [https://github.com/vallis/libstempo](https://github.com/vallis/libstempo)
- **enterprise**: [https://github.com/nanograv/enterprise](https://github.com/nanograv/enterprise)
- **enterprise_extensions**: [https://github.com/nanograv/enterprise_extensions](https://github.com/nanograv/enterprise_extensions)
- **PTMCMCSampler**: [https://github.com/jellis18/PTMCMCSampler](https://github.com/jellis18/PTMCMCSampler)
- **matplotlib**: [https://github.com/matplotlib/matplotlib](https://github.com/matplotlib/matplotlib)
- **corner**: [https://github.com/dfm/corner.py](https://github.com/dfm/corner.py)

In [ ]:
from IPython.display import display, HTML

# This will adjust the output cell width in Google Colab
display(HTML("<style>.output-cell { max-width: 90% !important; }</style>"))

In [ ]:
# Standard library
import os

# Scientific computing and linear algebra
import numpy as np
import scipy.linalg as sl

# libstempo library
import libstempo as LT
import libstempo.toasim as LTsim
from libstempo.toasim import (
    add_efac, 
    add_equad, 
    add_gwb, 
    add_cgw
)

# enterprise and extensions
from enterprise.pulsar import Pulsar
from enterprise.signals.utils import powerlaw, createfourierdesignmatrix_dm
from enterprise.signals import white_signals, gp_signals, parameter
from enterprise.signals.signal_base import PTA
from enterprise_extensions.sampler import JumpProposal

# PTMCMCSampler
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

# Plotting libraries
import matplotlib.pyplot as plt
import corner

# Import pickle
import pickle

In [ ]:
# Constants
DAY = 24 * 3600
YEAR = 365.25 * DAY

def add_efac(psr, efac=1.0, flagid=None, flags=None, seed=None):
    """
    Adapted from libstempo.
    Add nominal TOA errors, multiplied by the `efac` factor.
    Optionally, use a pseudorandom-number-generator seed.
    """
    if seed is not None:
        np.random.seed(seed)

    # Default efacvec
    efacvec = np.ones(psr.nobs)

    if flags is None:
        if not np.isscalar(efac):
            raise ValueError("If flags is None, efac must be a scalar.")
        efacvec = np.full(psr.nobs, efac)
    elif flagid is not None and not np.isscalar(efac):
        if len(efac) == len(flags):
            flagvals = np.array(psr.flagvals(flagid))
            for ct, flag in enumerate(flags):
                efacvec[flagvals == flag] = efac[ct]

    # Add TOA errors with random noise
    psr.stoas[:] += efacvec * psr.toaerrs * (1e-6 / DAY) * np.random.randn(psr.nobs)

def add_time_corr_signal(psr, A, gamma, components=10, tspan=None, seed=None, idx=0, factor=1.0):
    """
    Taken from libstempo.
    Add DM variations with P(f) = A^2 / (12 pi^2) (f year)^-gamma using Fourier bases.
    Optionally use a pseudorandom-number-generator seed.
    """
    if seed is not None:
        np.random.seed(seed)

    t = psr.toas()
    fref = 1400
    v = (fref / psr.freqs) ** idx

    minx, maxx = np.min(t), np.max(t)
    if tspan is None:
        x = (t - minx) / (maxx - minx)
        T = (DAY / YEAR) * (maxx - minx)
    else:
        x = (t - minx) / tspan
        T = (DAY / YEAR) * tspan

    size = 2 * components
    F = np.zeros((psr.nobs, size))
    f = np.zeros(size)

    # Vectorized computation of Fourier components
    i_vals = np.arange(1, components + 1)
    i_vals = i_vals[:, np.newaxis]  # Shape (30, 1)
    x = np.asarray(x)  # Ensure x is a 1D array
    F[:, ::2] = np.cos(2 * np.pi * i_vals * x).T  # Cosine terms
    F[:, 1::2] = np.sin(2 * np.pi * i_vals * x).T  # Sine terms

    f[::2] = f[1::2] = np.squeeze(i_vals / T)

    norm = A**2 * YEAR**2 / (12 * np.pi**2 * T)
    prior = norm * f ** (-gamma)

    y = np.sqrt(prior) * np.random.randn(size)
    psr.stoas[:] += (1.0 / DAY) * np.dot(F, y) * v * factor


### Explanation

This code defines two functions, `add_efac` and `add_time_corr_signal`, to add errors and signals to the Times of Arrival (TOA) of pulsar observations. 

### Constants

The constants are defined for time conversions:
\[
\text{DAY} = 24 \times 3600 = 86400 \, \text{seconds}
\]
\[
\text{YEAR} = 365.25 \times \text{DAY} = 31,557,600 \, \text{seconds}
\]

### `add_efac` Function

The `add_efac` function is designed to add errors to the TOA measurements of a pulsar. Here's a breakdown of its components:

#### Parameters:
- **psr**: A pulsar object that contains observations and related properties.
- **efac**: A scalar (or vector if using flags) that multiplies the nominal TOA errors.
- **flagid** and **flags**: These parameters allow customization of the error factor (efac) for different subsets of data based on flags.
- **seed**: A random seed to ensure reproducibility of the random errors.

#### Steps:
1. **Random Seed**: If a `seed` is provided, the random number generator is initialized with it.
2. **Error Factor Vector (`efacvec`)**: 
   - If no flags are provided, `efac` must be a scalar. The function creates an array `efacvec` of size equal to the number of observations (`psr.nobs`) and fills it with the scalar value of `efac`.
   - If flags are provided, the `efacvec` array is adjusted based on which observations match the provided flags.
3. **Add TOA Errors**:
   - Random noise is generated using a normal distribution with mean 0 and standard deviation 1, scaled by `psr.toaerrs`, `efacvec`, and a constant \(\frac{1}{\text{DAY}} \times 10^{-6}\):
     \[
     \text{psr.stoas} += \text{efacvec} \times \text{psr.toaerrs} \times \left( \frac{10^{-6}}{\text{DAY}} \right) \times \text{np.random.randn(psr.nobs)}
     \]
   - This effectively adds Gaussian noise to the TOA values.

### `add_time_corr_signal` Function

This function adds a time-correlated signal to the pulsar's TOA data based on a Fourier representation of the signal. The function models variations in the dispersion measure (DM) using a power-law spectrum.

#### Parameters:
- **psr**: The pulsar object.
- **A**: Amplitude of the signal.
- **gamma**: Spectral index of the power-law.
- **components**: Number of Fourier components used to approximate the signal.
- **tspan**: Time span for the signal; if `None`, it's calculated from the data.
- **seed**: Random seed for reproducibility.
- **idx**: A scaling factor based on the observing frequency.
- **factor**: A general scaling factor for the added signal.

#### Steps:
1. **Random Seed**: If a seed is provided, the random number generator is initialized.
2. **Observation Times**: The TOA data (`psr.toas()`) is stored in `t`. Frequencies (`psr.freqs`) are scaled with a reference frequency, \(f_{\text{ref}} = 1400\) MHz, to obtain a scaling factor `v`:
   \[
   v = \left( \frac{f_{\text{ref}}}{\text{psr.freqs}} \right)^{\text{idx}}
   \]
3. **Normalized Time**: 
   - The observation times are normalized over the time span. If `tspan` is not provided, it is calculated as \( T = \frac{\text{DAY}}{\text{YEAR}} \times (\max(t) - \min(t)) \), where \( t \) is the TOA array. Otherwise, `tspan` is used to compute \( T \).
4. **Fourier Components**:
   - The Fourier design matrix `F` is constructed using cosine and sine terms for the first `components` Fourier modes:
     \[
     F[:, 2i] = \cos(2 \pi (i+1) x)
     \]
     \[
     F[:, 2i+1] = \sin(2 \pi (i+1) x)
     \]
   - The corresponding frequencies are stored in the array `f`, with the same frequency for both sine and cosine terms:
     \[
     f[2i] = f[2i+1] = \frac{i+1}{T}
     \]
5. **Power-Law Prior**:
   - The power-law prior is calculated based on the amplitude \(A\) and spectral index \(\gamma\):
     \[
     P(f) = \frac{A^2 \times \text{YEAR}^2}{12 \pi^2 \times T} \times f^{-\gamma}
     \]
   - A random realization of the signal is generated using these priors:
     \[
     y = \sqrt{P(f)} \times \text{np.random.randn(size)}
     \]
6. **Add Signal to TOA**:
   - Finally, the time-correlated signal is added to the TOA values:
     \[
     \text{psr.stoas} += \frac{1}{\text{DAY}} \times \text{np.dot}(F, y) \times v \times \text{factor}
     \]

### Summary of Equations

1. **TOA Error Addition**:
   \[
   \text{psr.stoas} += \text{efacvec} \times \text{psr.toaerrs} \times \left( \frac{10^{-6}}{\text{DAY}} \right) \times \mathcal{N}(0, 1)
   \]
   
2. **Time-Correlated Signal**:
   \[
   P(f) = \frac{A^2 \times \text{YEAR}^2}{12 \pi^2 \times T} \times f^{-\gamma}
   \]
   \[
   \text{psr.stoas} += \frac{1}{\text{DAY}} \times \text{np.dot}(F, y) \times v \times \text{factor}
   \]

# Simulate single-psr data

In [ ]:
psrname = "J0030+0451"
parfile = f"{homedir}/data/EPTA_DR2/DR2new/{psrname}/{psrname}.par"

cadence = 5.0  # days
# Calculate the number of observation points based on the total time range and cadence
start_mjd, end_mjd = 52000, 59000
num_toas = int((end_mjd - start_mjd) / cadence)

# Generate TOAs with proper spacing
toas = np.linspace(start_mjd, end_mjd, num_toas)  # MJD

toaerrs = 1  # microseconds
# Randomly assign frequencies from the available choices
freqs = np.random.choice([500, 900, 1400], num_toas)  # MHz

# Create the fake pulsar using libstempo
ltpsr = LTsim.fakepulsar(parfile, obstimes=toas, toaerr=toaerrs, freq=freqs)

## Ideal Timing model

In [ ]:
f, (a0, a1) = plt.subplots(1, 2, figsize=(22,7), 
                           gridspec_kw={'width_ratios': [3, 1], 'hspace': 0, 'wspace': 0}, 
                          )

# Left plot
a0.errorbar(ltpsr.toas(), ltpsr.residuals(), yerr=ltpsr.toaerrs*1e-6, fmt='.')

a0.axhline(0., ls=':', c='k', lw=2, zorder=10)
a0.grid(alpha=.4)

# Right plot
Gauss = np.random.randn(10000)
a1.hist(Gauss, bins=30, color='r', histtype='step', orientation="horizontal", density=True)
a1.hist(ltpsr.residuals() / (ltpsr.toaerrs*1e-6), 
        bins=30, histtype='step', orientation="horizontal", density=True)
plt.setp(a1.get_xticklabels(), visible=False)
plt.setp(a1.get_yticklabels(), visible=False)
a1.grid(alpha=.4)

plt.show()

In [ ]:
add_gwb(ltpsr, gwAmp = 2e-15, flow=1e-9, fhigh=1e-5)

In [ ]:
def UnixTimeFromModifiedJulianDate(jd):
    return (jd-40587)/365.25 + 1970;

chain_plot = plt.figure(figsize=(15, 5))
plt.axhline(0, color='k', linestyle='--')

toas = UnixTimeFromModifiedJulianDate(ltpsr.toas())
plt.plot(UnixTimeFromModifiedJulianDate(ltpsr.toas()[toas>2005]), ltpsr.residuals()[toas>2005]*1e6, label = '$R(t)_{Earth}$', color='k', linestyle='-')

plt.ylabel('Timing residuals ($\mu s$)', fontsize=14)
plt.xlabel('Year', fontsize=14)

In [ ]:
pickle.dump(ltpsr.residuals()*1e6, open('GWB_res.pkl', 'wb'))

In [ ]:
from enterprise_extensions import models, model_utils
from enterprise.pulsar import Pulsar

Tspan = 25.4 * 365.25 * 24 * 3600

# compute injected free spectrum
G = 6.6743e-8
M_solar = 1.98847e33
c = 2.99792458e10

df = 1/Tspan
N = 30
f_yr = 1 / (365.25 * 24 * 3600)

def compute_PSD(f, gamma, log10_A):
    return (10**(log10_A))**2/12/np.pi**2*(f/f_yr)**(-gamma)*(np.pi*10**7)**3

def hc_to_logrho(f_bin, hc, Tspan):
    return 0.5 * np.log10(hc**2 / 12 / np.pi**2 / f_bin**3 / Tspan)

def PSD_to_logrho(f_bin, PSD, Tspan):
    return  0.5*np.log10(PSD/Tspan)

f = np.linspace(1e-9, 1e-7, 1000)
PSD = compute_PSD(f, 13/3, np.log10(2e-15))
log_rho = PSD_to_logrho(f, PSD, Tspan)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15, 4), gridspec_kw={'width_ratios':[1, 2.5]})
fig.subplots_adjust(wspace=0.14)

axs[0].plot(np.log10(f), log_rho, color='tab:red')
#axs[0].xaxis.set_tick_params(labelsize=11)
#axs[0].yaxis.set_tick_params(labelsize=11)
axs[0].set_xlabel("Frequency [Hz]", fontsize=11)
axs[0].set_ylabel(r"$\mathrm{log}_{10}\left( \rho_{\mathrm{CRS}} \ [\mathrm{s}] \right)$", fontsize=11)

gridxs = np.log10(np.hstack((np.arange(1,10)*1e-9, np.arange(1,5)*1e-8)))
gridys = np.arange(-9.5,-5.5,.5)
labelslist = ['1$\cdot$10$^{-9}$', ' ',' ',' ',
              ' ',' ',' ',' ',
              ' ','1$\cdot$10$^{-8}$', ' ',' ',' ']
axs[0].set_xticks(gridxs, labels = labelslist)
axs[0].set_yticks(gridys)
axs[0].grid(ls='--')


axs[1].axhline(0, color='k', linestyle='--')

toas = UnixTimeFromModifiedJulianDate(ltpsr.toas())
axs[1].plot(UnixTimeFromModifiedJulianDate(ltpsr.toas()[toas>2005]), ltpsr.residuals()[toas>2005]*1e6, label = '$R(t)_{Earth}$', color='k', linestyle='-')

axs[1].set_ylabel('Timing residuals ($\mu s$)', fontsize=11)
axs[1].set_xlabel('Year', fontsize=11)

plt.savefig('res_gwb_example.png', dpi=150, bbox_inches='tight')

## Inject White noise

In [ ]:
res_i = np.copy(ltpsr.residuals())
efac = 1.
add_efac(ltpsr, efac=efac, seed=9827)
res_wn = np.copy(ltpsr.residuals()) - res_i

In [ ]:
f, (a0, a1) = plt.subplots(1, 2, figsize=(22,7), 
                           gridspec_kw={'width_ratios': [3, 1], 'hspace': 0, 'wspace': 0}, 
                          )
# Left plot
a0.errorbar(ltpsr.toas(), ltpsr.residuals(), yerr=ltpsr.toaerrs*1e-6, fmt='.')
a0.axhline(0., ls=':', c='k', lw=2, zorder=10)
a0.grid(alpha=.4)

# Right plot
Gauss = np.random.randn(10000)
a1.hist(Gauss, bins=30, color='r', histtype='step', orientation="horizontal", density=True)
a1.hist(ltpsr.residuals() / (ltpsr.toaerrs*1e-6), 
        bins=30, histtype='step', orientation="horizontal", density=True)
plt.setp(a1.get_xticklabels(), visible=False)
plt.setp(a1.get_yticklabels(), visible=False)
a1.grid(alpha=.4)

plt.show()

## Inject red noise

In [ ]:
# Set up random values for A_rn and gamma_rn
A_rn = 7e-14 #np.random.uniform(1e-15, 1e-13)
gamma_rn = 3 #np.random.uniform(1, 5)

# Copy initial residuals
toa_tmp = np.copy(ltpsr.residuals())

# Add time-correlated signal with random A_rn and gamma_rn
add_time_corr_signal(ltpsr, A=A_rn, gamma=gamma_rn, components=30, seed=4105)

# Compute new residuals after adding red noise
res_rn = np.copy(ltpsr.residuals()) - toa_tmp

# Print the values in table format
print(f"{'Parameter':<10} | {'Value':<18}")
print("-" * 30)
print(f"{'A_rn':<10} | {A_rn:<18.5e}")
print(f"{'gamma_rn':<10} | {gamma_rn:<18.5f}")

In [ ]:
f, (a0, a1) = plt.subplots(1, 2, figsize=(22,7), 
                           gridspec_kw={'width_ratios': [3, 1], 'hspace': 0, 'wspace': 0}, 
                          )

# Left plot
a0.errorbar(ltpsr.toas(), ltpsr.residuals(), yerr=ltpsr.toaerrs*1e-6, fmt='.', label="data")
a0.plot(ltpsr.toas(), res_rn, 'r', lw=3, zorder=10, label="Red noise")
a0.axhline(0., ls=':', c='k', lw=2, zorder=10)
a0.grid(alpha=.4)
a0.legend(fontsize=14)


# Right plot
Gauss = np.random.randn(10000)
# Calculate the normalized residuals
normalized_residuals = ltpsr.residuals() / (ltpsr.toaerrs*1e-6)
# Remove NaNs from the data
normalized_residuals = normalized_residuals[~np.isnan(normalized_residuals)]
a1.hist(Gauss, bins=30, color='r', histtype='step', orientation="horizontal", density=True)
a1.hist(normalized_residuals, 
        bins=30, histtype='step', orientation="horizontal", density=True)
plt.setp(a1.get_xticklabels(), visible=False)
plt.setp(a1.get_yticklabels(), visible=False)
a1.grid(alpha=.4)

plt.show()

## Inject DM variations

In [ ]:
A_dm = 5e-14
gamma_dm = 2.3
toa_tmp = np.copy(ltpsr.residuals())
add_time_corr_signal(ltpsr, A=A_dm, gamma=gamma_dm, components=30, idx=2, seed=413405)
res_dm = np.copy(ltpsr.residuals()) - toa_tmp

In [ ]:
f, (a0, a1) = plt.subplots(1, 2, figsize=(22,7), 
                           gridspec_kw={'width_ratios': [3, 1], 'hspace': 0, 'wspace': 0}, 
                          )

# Left plot
a0.errorbar(ltpsr.toas(), ltpsr.residuals(), yerr=ltpsr.toaerrs*1e-6, fmt='.', label='Data')
a0.plot(ltpsr.toas(), res_rn, 'r', lw=3, zorder=10, label="Red noise")
a0.plot(ltpsr.toas(), res_dm, '.', color='g', lw=3, zorder=10, label="DM var.")
a0.axhline(0., ls=':', c='k', lw=2, zorder=10)
a0.grid(alpha=.4)
a0.legend(fontsize=14)

# Right plot
Gauss = np.random.randn(10000)
a1.hist(Gauss, bins=30, color='r', histtype='step', orientation="horizontal", density=True)
a1.hist(ltpsr.residuals() / (ltpsr.toaerrs*1e-6), 
        bins=30, histtype='step', orientation="horizontal", density=True)
plt.setp(a1.get_xticklabels(), visible=False)
plt.setp(a1.get_yticklabels(), visible=False)
a1.grid(alpha=.4)

plt.show()

# Create model with enterprise

In [ ]:
# Create Enteprise pulsar object
psr = Pulsar(ltpsr)

## Include TM param errors (marginalized)

In [ ]:
s = gp_signals.TimingModel()

## Include WN EFAC

In [ ]:
efac_prior = parameter.Uniform(0.1, 5)
s += white_signals.MeasurementNoise(efac=efac_prior)

## Include Red noise

In [ ]:
log10_A_rn_prior = parameter.Uniform(-18, -10)
gamma_rn_prior = parameter.Uniform(0, 7)
pl = powerlaw(log10_A=log10_A_rn_prior, gamma=gamma_rn_prior)
s += gp_signals.FourierBasisGP(pl, components=30)

## Include DM variations

In [ ]:
log10_A_dm_prior = parameter.Uniform(-18, -10)
gamma_dm_prior = parameter.Uniform(0, 7)
pl = powerlaw(log10_A=log10_A_rn_prior, gamma=gamma_rn_prior)
dm_basis = createfourierdesignmatrix_dm(nmodes=30)
s += gp_signals.BasisGP(pl, dm_basis, name="dm_gp")

In [ ]:
# Create PTA object, from which one can compute LH and prior
pta = PTA([s(psr)])

In [ ]:
for p in pta.param_names:
    print(p)

In [ ]:
x = {
    f"{psrname}_dm_gp_gamma" : 3,
    f"{psrname}_dm_gp_log10_A" : -14,
    f"{psrname}_efac" : 1,
    f"{psrname}_red_noise_gamma" : 3,
    f"{psrname}_red_noise_log10_A" : -15
}
print(pta.get_lnlikelihood(x))
print(pta.get_lnprior(x))

# Set sampler

In [ ]:
# Sample parameters
outdir = "./../chains/sglpsr"
nsamples = 1e4

with open(os.path.join(outdir, "pars.txt"), "w") as fout:
    for pname in pta.param_names:
        fout.write(pname + "\n")

x0 = np.hstack([p.sample() for p in pta.params])
ndim = len(x0)
cov = np.diag(np.ones(ndim) * 0.01**2)
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, outDir=outdir)
## to resume black txt

In [ ]:
# Set Jump Proposals
jp = JumpProposal(pta, None, empirical_distr=None)

# always add draw from prior
sampler.addProposalToCycle(jp.draw_from_prior, 5)

sel_sig = {"red_noise":10, "dm":10}

# Jump proposals from priors of selected params
for s in sel_sig:
    if any([s in p for p in pta.param_names]):
        sampler.addProposalToCycle(jp.draw_from_par_prior(s), 10)

 # Sample parameters

In [ ]:
sampler.sample(x0, int(nsamples), SCAMweight=40, DEweight=60, AMweight=20)

# Analyse chain

In [ ]:
ch = np.loadtxt("%s/chain_1.txt"%outdir)
ch = ch[int(len(ch)*.5):]
pars = np.loadtxt("%s/pars.txt"%outdir, dtype=str)
p_inj = [gamma_dm, np.log10(A_dm), efac, gamma_rn, np.log10(A_rn)]

In [ ]:
for p in pars:
    print(p)

In [ ]:
for i in range(len(pars)):
    plt.title(pars[i])
    plt.plot(ch[:,i])
    plt.axhline(p_inj[i], c='k', lw=3)
    plt.show()

In [ ]:
corner.corner(ch[::2,:-4], labels=pars, truths=p_inj, truth_color='k', hist_kwargs={"density":True})
plt.show()